* 每次验证结果的时候都重新分割一次测试数据集实在是太浪费时间了，于是写一段脚本来完成分割，注意kaggle里面是不能批量下载生成的文件的，所以我还是建议你本地执行这个脚本，然后上传成为自己的数据集，以便使用。

In [ ]:
'''
import os
import SimpleITK
import numpy as np
import cv2
from tqdm import tqdm
import shutil
 
def convert_from_dicom_to_jpg(img,low_window,high_window,save_path):
    lungwin = np.array([low_window*1.,high_window*1.])
    newimg = (img-lungwin[0])/(lungwin[1]-lungwin[0])    #归一化
    newimg = (newimg*255).astype('uint8')                #将像素值扩展到[0,255]
    stacked_img = np.stack((newimg,) * 3, axis=-1)
    cv2.imwrite(save_path, stacked_img, [int(cv2.IMWRITE_JPEG_QUALITY), 100])
 
if __name__ == '__main__':
    #dicom文件目录
    dicom_dir = '../input/siim-covid19-detection/test/'
 
    path = "./test/study/"
    if os.path.exists(path):
        shutil.rmtree(path)
    os.makedirs(path)
    picsum = 0
    for dirname, _, filenames in tqdm(os.walk(dicom_dir)):
        for i in filenames:
            picsum += 1
            dcm_image_path = os.path.join(dirname,i)  # 读取dicom文件
            name, _ = os.path.splitext(i)
            output_jpg_path = os.path.join(path, name+'.png')
            ds_array = SimpleITK.ReadImage(dcm_image_path)  # 读取dicom文件的相关信息
            img_array = SimpleITK.GetArrayFromImage(ds_array)  # 获取array
            # SimpleITK读取的图像数据的坐标顺序为zyx，即从多少张切片到单张切片的宽和高，此处我们读取单张，因此img_array的shape
            # 类似于 （1，height，width）的形式
            shape = img_array.shape
            img_array = np.reshape(img_array, (shape[1], shape[2]))  # 获取array中的height和width
            high = np.max(img_array)
            low = np.min(img_array)
            convert_from_dicom_to_jpg(img_array, low, high, output_jpg_path)  # 调用函数，转换成jpg文件并保存到对应的路径
    print('图片总数：', picsum)
'''

In [ ]:
!conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y

In [ ]:
import os

from PIL import Image
import pandas as pd
from tqdm.auto import tqdm

import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data


def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [ ]:
split = 'test'
save_dir = f'./{split}/'

os.makedirs(save_dir, exist_ok=True)

save_dir = f'./{split}/study/'
os.makedirs(save_dir, exist_ok=True)

picsum = 0
for dirname, _, filenames in tqdm(os.walk(f'../input/siim-covid19-detection/{split}')):
    for file in filenames:
        picsum += 1
        #print(picsum)
        # set keep_ratio=True to have original aspect ratio
        xray = read_xray(os.path.join(dirname, file))
        im = resize(xray, size=512)  
        study = dirname.split('/')[-2] + '_study.png'
        #study = dirname.split('/')[-2] + '_study'
        im.save(os.path.join(save_dir, study))
print(picsum)